In [14]:
import pandas as pd
import numpy as np
from math import cos, sin, pi, hypot
from itertools import combinations

# ---------- Params ----------
VEH_FILE = "vehicles.csv"
INST_FILE = "./instances/instance_10.csv"
OUT_FILE = "routes.csv"

R_earth = 6.371e6
deg2rad = pi / 180.0
T_DAY = 86400.0
omega = 2 * pi / T_DAY
tdep = 0.0  # depot departure

# ---------- I/O ----------
vehicles = pd.read_csv(VEH_FILE)
inst = pd.read_csv(INST_FILE)

# Identify depot (id == 0 expected)
depot = inst[inst['id'] == 0].iloc[0]
customers_df = inst[inst['id'] != 0].copy()
customers_df = customers_df.reset_index(drop=True)

# convert lat/lon degrees -> meters using standard approximation:
lat0 = depot['latitude'] * deg2rad
def deg_to_meters(lon_deg, lat_deg):
    x = R_earth * (lon_deg * deg2rad) * np.cos(lat0)
    y = R_earth * (lat_deg * deg2rad)
    return float(x), float(y)

inst['x_m'], inst['y_m'] = zip(*inst.apply(lambda r: deg_to_meters(r['longitude'], r['latitude']), axis=1))
coords = inst.set_index('id')[['x_m','y_m']].T.to_dict()

# dictionary params for customers
weights = inst.set_index('id')['order_weight'].fillna(0).to_dict()
tmin = inst.set_index('id')['window_start'].fillna(-1e9).to_dict()
tmax = inst.set_index('id')['window_end'].fillna(1e12).to_dict()
ldur = inst.set_index('id')['delivery_duration'].fillna(0).to_dict()

node_ids = inst['id'].tolist()
customer_ids = [int(i) for i in inst['id'] if i != 0]

# ---------- Distances ----------
def manhattan(i, j):
    xi, yi = coords[i]['x_m'], coords[i]['y_m']
    xj, yj = coords[j]['x_m'], coords[j]['y_m']
    return abs(xj - xi) + abs(yj - yi)

def euclidean(i, j):
    xi, yi = coords[i]['x_m'], coords[i]['y_m']
    xj, yj = coords[j]['x_m'], coords[j]['y_m']
    return hypot(xj - xi, yj - yi)

deltaM = {(i,j): manhattan(i,j) for i in node_ids for j in node_ids if i!=j}
deltaE = {(i,j): euclidean(i,j) for i in node_ids for j in node_ids if i!=j}

# ---------- Travel time model ----------
# gamma_f(t) from fourier cos/sin coefficients
def gamma_of(family_row, t):
    # family_row: a pandas Series with columns fourier_cos_0..3 and fourier_sin_0..3
    s = 0.0
    for n in range(4):
        s += family_row[f"fourier_cos_{n}"] * np.cos(n * omega * t)
        s += family_row[f"fourier_sin_{n}"] * np.sin(n * omega * t)
    return float(s)

def travel_time(family_row, i, j, depart_time):
    # reference travel time: deltaM / speed + parking_time
    ref = deltaM[(i,j)] / float(family_row['speed']) + float(family_row['parking_time'])
    gamma = gamma_of(family_row, depart_time % T_DAY)
    # impose non-negative gamma (in case data noise) and at least small epsilon
    if gamma < 1e-6:
        gamma = 1e-6
    return ref * gamma

# ---------- Route simulation / feasibility ----------
def simulate_route_times(route_list, family_row):
    # route_list: sequence of customer ids (not including depots)
    # returns (feasible:bool, arrivals dict id->a, departures dict id->d, total_manhattan, radius)
    arrivals = {}
    departures = {}
    cur_depart = tdep  # depart depot at tdep
    prev = 0
    total_man = 0.0

    for cust in route_list:
        # travel from prev -> cust with depart time cur_depart
        if prev == cust:
            return False, None, None, None, None
        if prev != cust:
            if (prev, cust) not in deltaM:
                return False, None, None, None, None
            tt = travel_time(family_row, prev, cust, cur_depart)
            total_man += deltaM[(prev,cust)]
            arrival = cur_depart + tt
        # waiting allowed until tmin
        if arrival < tmin[cust]:
            arrival = tmin[cust]
        # check window
        if arrival > tmax[cust] + 1e-9:
            return False, None, None, None, None
        start_service = arrival
        dep = start_service + ldur[cust]
        arrivals[cust] = start_service
        departures[cust] = dep
        cur_depart = dep
        prev = cust

    # finally return to depot
    if prev != 0:
        if (prev,0) in deltaM:
            total_man += deltaM[(prev,0)]
            # compute travel back (not checked against window)
            _ = travel_time(family_row, prev, 0, cur_depart)

    # compute radius: .5 * max pairwise euclidean among customers in route
    if len(route_list) <= 1:
        radius = 0.0
    else:
        maxp = 0.0
        for i,j in combinations(route_list, 2):
            d = deltaE[(i,j)]
            if d > maxp:
                maxp = d
        radius = 0.5 * maxp

    return True, arrivals, departures, total_man, radius

# ---------- Heuristic insertion ----------
# We'll process customers in order of earliest tmin (greedy)
order_list = sorted(customer_ids, key=lambda i: tmin[i])

routes = []  # list of dicts: {'customers':[...], 'family': f_id, 'load':..., 'cost':...}
# Each route will keep customers in current order; we will attempt to insert new customer into best position.

for cust in order_list:
    inserted = False
    # Try each existing route and each insertion position; test all families for feasibility.
    best_option = None  # (route_index, pos, family_row, cost_increase, new_route_info)
    for r_idx, route in enumerate(routes):
        base_customers = route['customers']
        # try insertion at every position 0..len
        for pos in range(len(base_customers)+1):
            new_customers = base_customers[:pos] + [cust] + base_customers[pos:]
            # For this tentative route, try all families that have capacity
            for _, fam in vehicles.iterrows():
                cap = fam['max_capacity']
                new_load = route['load'] + weights[cust]
                if new_load > cap + 1e-9:
                    continue
                feasible, arr, dep, total_man, radius = simulate_route_times(new_customers, fam)
                if not feasible:
                    continue
                # compute cost of this tentative route (rental counted once per route)
                # We estimate incremental cost: new route cost - base route cost
                # compute base cost components
                # base route cost may not be computed for family change; for simplicity, compute full cost and compare
                rental = fam['rental_cost']
                fuel = fam['fuel_cost'] * total_man
                radius_cost = fam['radius_cost'] * radius
                total_cost = rental + fuel + radius_cost
                # base route if same family? we will compare absolute cost and decide best
                # store option
                if best_option is None or total_cost < best_option[3]:
                    best_option = (r_idx, pos, fam, total_cost, new_customers, new_load, total_man, radius, arr, dep)
    # If found a best_option, apply it (choose route & family that minimizes the route cost)
    if best_option:
        r_idx, pos, fam, tot_cost, new_customers, new_load, total_man, radius, arr, dep = best_option
        # Update existing route: set family to fam, customers to new_customers, load, cost
        routes[r_idx]['customers'] = new_customers
        routes[r_idx]['family'] = fam['family']
        routes[r_idx]['load'] = new_load
        routes[r_idx]['manhattan'] = total_man
        routes[r_idx]['radius'] = radius
        routes[r_idx]['cost'] = tot_cost
        routes[r_idx]['arrivals'] = arr
        routes[r_idx]['departures'] = dep
        inserted = True

    if not inserted:
        # open a new route for this customer: choose best family that can serve single customer (feasible)
        best_f = None
        best_cost = None
        best_info = None
        for _, fam in vehicles.iterrows():
            if weights[cust] > fam['max_capacity'] + 1e-9:
                continue
            feasible, arr, dep, total_man, radius = simulate_route_times([cust], fam)
            if not feasible:
                continue
            rental = fam['rental_cost']
            fuel = fam['fuel_cost'] * total_man
            radius_cost = fam['radius_cost'] * radius
            tot_cost = rental + fuel + radius_cost
            if best_cost is None or tot_cost < best_cost:
                best_cost = tot_cost
                best_f = fam
                best_info = (arr, dep, total_man, radius)
        if best_f is None:
            raise RuntimeError(f"No feasible family can serve customer {cust} alone: instance infeasible or data inconsistent.")
        # add new route
        arr, dep, total_man, radius = best_info
        routes.append({
            'customers': [cust],
            'family': best_f['family'],
            'load': weights[cust],
            'manhattan': total_man,
            'radius': radius,
            'cost': best_cost,
            'arrivals': arr,
            'departures': dep
        })

# ---------- Final evaluation ----------
total_cost = 0.0
detailed = []
for r in routes:
    # recompute final route cost using assigned family row
    fam_row = vehicles[vehicles['family'] == r['family']].iloc[0]
    # simulate again to get manhattan/travel
    feasible, arr, dep, total_man, radius = simulate_route_times(r['customers'], fam_row)
    if not feasible:
        # Shouldn't happen, but mark heavy penalty
        print("Warning: route became infeasible during final evaluation.")
        route_cost = 1e9
    else:
        route_cost = fam_row['rental_cost'] + fam_row['fuel_cost'] * total_man + fam_row['radius_cost'] * radius
    total_cost += route_cost
    detailed.append({
        'family': int(r['family']),
        'customers': r['customers'],
        'load': r['load'],
        'manhattan': total_man,
        'radius': radius,
        'cost': route_cost,
        'feasible': feasible
    })

In [15]:
total_cost

np.float64(13591.335495192146)

In [17]:
len(detailed)

41

In [ ]:
for r in detailed :
    print(r['family'], r['customers'])
    

1 [4, 15, 19, 161, 12, 5, 17, 16, 18, 2, 254, 190, 266, 406, 168, 21]
2 [92, 24, 94, 64, 81, 28, 61, 22, 159, 221, 57, 54, 40, 63, 105, 96, 475, 270, 371, 228]
3 [110, 131, 128, 133, 136, 114, 115, 108, 155, 142, 140, 303, 271, 278, 368, 146, 438, 273, 241, 20, 347]
1 [194, 165, 182, 175, 203, 195, 174, 188, 162, 325, 283, 251, 213, 204, 281, 260, 205]
1 [224, 247, 264, 225, 218, 234, 246, 238, 327, 245, 259, 433]
1 [297, 289, 309, 288, 321, 312, 311, 310, 319, 169, 317, 328]
1 [332, 354, 340, 329, 335, 345, 341, 350, 478, 41, 358, 222, 407, 184, 355]
1 [395, 380, 474, 400, 389, 366, 370, 363, 37, 473, 482, 14, 83, 374, 494, 463, 149, 418]
1 [414, 454, 465, 441, 458, 415, 443, 402, 472, 440, 364, 486, 49, 382, 50]
1 [118, 299, 292, 391, 492, 262, 58, 7, 3, 206, 45, 480, 483, 44, 462]
1 [97, 116, 91, 88, 72, 132, 137, 426, 196, 153, 344, 144, 352, 412]
1 [172, 166, 217, 199, 193, 156, 211, 210, 223, 381, 333, 313, 453, 437, 229, 455, 300, 353]
1 [243, 287, 239, 265, 233, 286, 232, 394, 